In [2]:
pip install pymed

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [11]:
import nucleus_api
import os
import csv
import json
import datetime
import time
from nucleus_api.rest import ApiException
import nucleus_api.api.nucleus_api as nucleus_helper
from pprint import pprint
import numpy as np
from pathlib import Path
from pymed import PubMed
from lxml import etree
from io import StringIO, BytesIO

configuration = nucleus_api.Configuration()
# configuration.host = 'localhost:5000'
configuration.host = 'http://3.91.117.194:5000/'
configuration.api_key['x-api-key'] = 'YTRgtK8T2H-hhR5NbmKW3w'

api_instance = nucleus_api.NucleusApi(nucleus_api.ApiClient(configuration))

In [14]:
dataset = 'PubMedDataset2'
# nucleus.Dataset.delete_dataset(dataset)
pubmed = PubMed(tool="NLPLahont", email="21tuong.dc")
pubmed.parameters["db"] = "pmc" #pubmed have no text so pmc is my new best friend

results_ids =  pubmed._getArticleIds(query="[DOI]", max_results=500)
count = 0

for i in range(100):
    pubmed.parameters["db"] = "pmc" #pubmed have no text so pmc is my new best friend
    parameters = pubmed.parameters.copy()
    parameters["id"] = results_ids[5*i:5*i+4]
    print(parameters)
    response = pubmed._get(url="/entrez/eutils/efetch.fcgi", parameters=parameters, output="xml")
    # print(response)
    root = etree.parse(StringIO(response))

    for article in root.iter("article"):
        newdocument = {}
        #getting title
        newdocument["title"] = article.xpath('(.//article-title)')[0].text
        #getting publication date
        day = article.xpath('(.//pub-date/day)')[0].text
        month = article.xpath('(.//pub-date/month)')[0].text
        year = article.xpath('(.//pub-date/year)')[0].text
        newdocument["time"] = year + "-" + month + "-" + day 
        
        #extracting content
        content_results = article.xpath('(.//p|.//p/xref|.//p/sup|.//p/i|.//p/u|.//p/b)')
        content = " ".join([str(text) for tup in [(sub.text, sub.tail) for sub in content_results] for text in tup])
        newdocument["content"] = " ".join(content.split())
        count += 1
        print(newdocument)
        # load into dataset
        payload = nucleus_api.Appendjsonparams(dataset=dataset,
                                           document=newdocument)
        try:
            api_response = api_instance.post_append_json_to_dataset(payload)
            print(api_response.result)
        except ApiException as e:
            api_error = json.loads(e.body)
            print('ERROR:', api_error['message'])

print(i)

SSLError: HTTPSConnectionPool(host='eutils.ncbi.nlm.nih.gov', port=443): Max retries exceeded with url: /entrez/eutils/esearch.fcgi?tool=NLPLahont&email=21tuong.dc%40vinuni.edu.vn&db=pmc&term=%5BDOI%5D&retmax=500&retmode=json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1125)')))

In [3]:
query = ''
custom_stop_words = [] # str | List of stop words. (optional)
num_topics = 5 # int | Number of topics to be extracted from the dataset. (optional) (default to 8)
num_keywords = 4 # int | Number of keywords per topic that is extracted from the dataset. (optional) (default to 8)
summary_length = 3 # int | The maximum number of bullet points a user wants to see in each topic summary. (optional) (default to 6)
context_amount = 0 # int | The number of sentences surrounding key summary sentences in the documents that they come from. (optional) (default to 0)
num_docs = 20 # int | The maximum number of key documents to use for summarization. (optional) (default to 20)
excluded_docs = '' # str | List of document IDs that should be excluded from the analysis. Example, ["docid1", "docid2", ..., "docidN"]  (optional)
remove_redundancies = True # bool | If True, this option removes quasi-duplicates from the analysis. A quasi-duplicate would have the same NLP representation, but not necessarily the exact same text. (optional) (default True)
metadata_selection = ""
try:
    payload = nucleus_api.TopicSummaryModel	(dataset=dataset,                                
                                 query=query,
                                 summary_length = summary_length,
                                 num_keywords = 8,                 
                                 custom_stop_words=custom_stop_words,     
                                 num_topics=num_topics,
                                 metadata_selection=metadata_selection,
                                 remove_redundancies=remove_redundancies,
                                 time_period = "5Y"
)
    api_response = api_instance.post_topic_summary_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:
    topic_summarization = []
    for i,res in enumerate(api_response.result):
        print('Topic', i, 'summary:')
        print('    Keywords:', res.keywords)
        summarization = ""
        for j in range(len(res.summary)):
            print(res.summary[j])
            summarization += res.summary[j].sentences[2:-2]
        print(summarization)
        topic_summarization.append(summarization)
        print('---------------')
print(topic_summarization)

Topic 0 summary:
    Keywords: figure figure;figure supplement;cck neurons;lpb cck;revised figure;wild type;activation lpb;supplement figure
{'attribute': {'counts': None, 'time': 1678060800},
 'doc_id': '1053371824425218591',
 'sentences': "['These results were confirmed by RT-qPCR analyses performed on "
              'gene subsets from independent biological replicates ( Figure 2C '
              ", Figure 2—figure supplement 1B ).', 'We next applied "
              'high-resolution RNA-fluorescence in situ hybridization '
              '(RNA-FISH) to visualize pCharme alone ( Figure 3A , Figure '
              '3—figure supplement 1B ) or relatively to MATR3 through '
              'combined immunofluorescence IF/RNA-FISH approaches ( Figure 3B '
              ").']",
 'title': 'The long noncoding RNA Charme supervises cardiomyocyte maturation '
          'by controlling cell differentiation programs in the developing '
          'heart'}
{'attribute': {'counts': None, 'time': 1679